### Use `RGBD` camera

In [1]:
import mujoco,cv2,pyvista
import numpy as np
import matplotlib.pyplot as plt
# from pyntcloud import PyntCloud
import sys
sys.path.append('../../')
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import sample_xyzs,rpy2r,r2rpy,compute_view_params,get_interp_const_vel_traj
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


/home/rilab/.pyenv/versions/3.8.13/envs/llm/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Parse `UR5e` with `RG2` gripper

In [7]:
xml_path = '../../asset/ur5e/scene_ur5e_rg2_d435i_obj_realworld.xml'
# xml_path = '../../asset/ur5e/scene_ur5e_rg2_d435i_vbm.xml'
env = MuJoCoParserClass(name='UR5e with RG2 gripper',rel_xml_path=xml_path,VERBOSE=True)

obj_cylinder_names = [body_name for body_name in env.body_names
             if body_name is not None and (body_name.startswith("obj_cylinder"))]
obj_box_names = [body_name for body_name in env.body_names
                if body_name is not None and (body_name.startswith("obj_box"))]
obj_glass_names = [body_name for body_name in env.body_names
                if body_name is not None and (body_name.startswith("obj_wine"))]

n_cylinder_obj = 3 # len(obj_cylinder_names)
n_box_obj = 3 # len(obj_box_names)
n_glass_obj = 3 # len(obj_glass_names)

# Place objects
env.place_objects(n_obj=3, obj_names=obj_cylinder_names, x_range=[0.75, 0.85], y_range=[-0.15, 0.15], COLORS=True, VERBOSE=True)
# env.place_objects(n_obj=3, obj_names=obj_box_names, COLORS=True, VERBOSE=True) 
env.place_objects(n_obj=3, obj_names=obj_glass_names, COLORS=False, VERBOSE=True)

# Move tables and robot base
env.model.body('base_table').pos = np.array([0,0,0])
env.model.body('front_object_table').pos = np.array([0.38+0.6,0,0])
env.model.body('right_object_table').pos = np.array([-0.05,-0.80,0])
env.model.body('left_object_table').pos = np.array([-0.05,0.80,0])
env.model.body('base').pos = np.array([0.18,0,0.79])


print ("Ready.")

dt:[0.0020] HZ:[500]
n_dof (=nv):[60]
n_geom:[62]
geom_names:['floor', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'front_object_table', 'right_object_table', 'left_object_table', 'base_table', 'obj_cylinder_01', 'obj_cylinder_02', 'obj_cylinder_03', 'obj_cylinder_04', 'obj_cylinder_05', 'obj_cylinder_06', 'obj_cylinder_07', 'obj_cylinder_08']
n_body:[31]
body_names:['world', 'base', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'tcp_link', 'camera_mount', 'd435i', 'rg2_gripper_base_link', 'camera_center', 'rg2_gripper_finger1_finger_link', 'rg2_gripper_finger1_inner_knuckle_link', 'rg2_gripper_finger1_finger_tip_link', 'rg2_gripper_finger2_finger_link', 'rg2_gripper_finger2_inner_knuckle

### Render

In [8]:
# Init viewer
env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=66.08,distance=3.0,elevation=-50,lookat=[0.4,0.18,0.71],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=False,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.25,jointwidth=0.05,jointrgba=[0.2,0.6,0.8,0.6])

# Base pose
body_name = 'tcp_link'
p_base = env.get_p_body(body_name='base')

# Straight pose
DO_RENDER_IK = True
p_trgt = p_base + np.array([0.4,0,0.6])
R_trgt = rpy2r(np.radians([0,0,0]))@rpy2r(np.radians([-180,0,90]))
q_prepose_00 = env.solve_ik(
    body_name=body_name,p_trgt=p_trgt,R_trgt=R_trgt,IK_P=True,IK_R=True,
    q_init=np.radians([0,-90,0,0,0,0]),idxs_forward=env.idxs_forward, idxs_jacobian=env.idxs_jacobian,
    RESET=False,DO_RENDER=DO_RENDER_IK,render_every=1,th=1*np.pi/180.0,err_th=1e-2)

# 30 deg down pose
p_trgt = p_base + np.array([0.4,0,0.6])
R_trgt = rpy2r(np.radians([0,30,0]))@rpy2r(np.radians([-180,0,90]))
q_prepose_30 = env.solve_ik(
    body_name=body_name,p_trgt=p_trgt,R_trgt=R_trgt,IK_P=True,IK_R=True,
    q_init=q_prepose_00,idxs_forward=env.idxs_forward, idxs_jacobian=env.idxs_jacobian,
    RESET=False,DO_RENDER=DO_RENDER_IK,render_every=1,th=1*np.pi/180.0,err_th=1e-4)

# 60 deg down pose
p_trgt = p_base + np.array([0.4,0,0.6])
R_trgt = rpy2r(np.radians([0,60,0]))@rpy2r(np.radians([-180,0,90]))
q_prepose_60 = env.solve_ik(
    body_name=body_name,p_trgt=p_trgt,R_trgt=R_trgt,IK_P=True,IK_R=True,
    q_init=q_prepose_30,idxs_forward=env.idxs_forward, idxs_jacobian=env.idxs_jacobian,
    RESET=False,DO_RENDER=DO_RENDER_IK,render_every=1,th=1*np.pi/180.0,err_th=1e-4)

env.close_viewer()
print ("IK done.")

# Get joint trajectory
times,q_traj = get_interp_const_vel_traj(
    np.vstack((q_prepose_60,q_prepose_30,q_prepose_30,q_prepose_60)),
    vel=np.radians(30),HZ=env.HZ)
print ("q_traj's L:[%d] time_max:[%.2f]sec"%(times.shape[0],times[-1]))


IK done.
q_traj's L:[1259] time_max:[2.52]sec


In [9]:
# Init viewer
env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=66.08,distance=3.0,elevation=-50,lookat=[0.4,0.18,0.71],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=False,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.25,jointwidth=0.05,jointrgba=[0.2,0.6,0.8,0.6])

pcd_tick = 0

# Reset
env.reset()
env.forward(q=q_traj[0,:],joint_idxs=[0,1,2,3,4,5])
tick,max_sec = 0,100
while env.get_sim_time() <= max_sec:
    # Step
    q = np.append(q_traj[tick,:],1.0) # open gripper
    env.step(ctrl=q,ctrl_idxs=[0,1,2,3,4,5,6])
    tick = min(tick + 1,q_traj.shape[0]-1)
    if not env.is_viewer_alive(): break
        
    # Render
    if env.loop_every(HZ=20):
        # Compute some poses
        p_tcp,R_tcp = env.get_pR_body(body_name='tcp_link')
        p_cam,R_cam = env.get_pR_body(body_name='camera_center')
        p_base,R_base = env.get_pR_body(body_name='base')
        # Get PCD from a specific view
        p_ego  = p_cam
        p_trgt = p_cam + R_cam[:,2]
        rgb_img,depth_img,pcd,xyz_img = env.get_egocentric_rgb_depth_pcd(
            p_ego=p_ego,p_trgt=p_trgt,rsz_rate=None,fovy=45,BACKUP_AND_RESTORE_VIEW=True)

        # Save PCD
        pcd_tick += 1
        # for p in pcd: env.plot_sphere(p=p,r=0.005,rgba=[0.95,0.05,0.05,1])
        env.render(render_every=1)
        
# Close viewer
env.close_viewer()
print ("Done. Tick:[%d] Time:[%.2f]sec"%(env.tick,env.get_sim_time()))

Pressed ESC
Quitting.
Done. Tick:[2052] Time:[4.10]sec


### `Passthrough` and `Downsample` the PCD

In [11]:
# passthrough filter about z axis
def passthrough_filter(pcd, axis, interval):
    mask = (pcd[:, axis] > interval[0]) & (pcd[:, axis] < interval[1])
    return pcd[mask]

pcd_masked = passthrough_filter(pcd, axis=2, interval=[0.5, 1.0])
pcd_masked_down = np.array(pcd_masked)[::50]  # downsample to visualize with markers for speed.
voxel_grid_down = point_cloud_to_voxel(pcd_masked_down, voxel_size)

### `Voxelize` pcd

In [12]:
import numpy as np

point_cloud_data = pcd_masked_down
voxel_size = 0.005  # size of each voxel

# Shift the point cloud to the origin and discretize it.
discrete_point_cloud = np.floor(point_cloud_data / voxel_size).astype(np.int32)
# Remove duplicates: this will give us the occupied voxels.
occupied_voxels = np.unique(discrete_point_cloud, axis=0)

In [13]:
import pyvista as pv

# Create a new unstructured grid
grid = pv.UnstructuredGrid()

# Add the voxels to the grid
for voxel in occupied_voxels:
    voxel = voxel * voxel_size  # convert back to original scale
    cube = pv.Cube(center=voxel, x_length=voxel_size, y_length=voxel_size, z_length=voxel_size)
    grid = grid.merge(cube)

# Visualize the voxel grid
grid.plot()


Widget(value="<iframe src='http://localhost:39695/index.html?ui=P_0x7ff70c0ce580_0&reconnect=auto' style='widt…

In [16]:
### pcd color information
pcd_masked_down_idx = np.round(pcd_masked_down[:, :2]).astype(np.int32)
pcd_masked_dowwn_color = rgb_img[pcd_masked_down_idx[:,0], pcd_masked_down_idx[:,1], :]
pcd_masked_dowwn_color = pcd_masked_dowwn_color.astype(np.float32) / 255.0

pcd_masked_down_pcdrgb = np.concatenate([pcd_masked_down, pcd_masked_dowwn_color], axis=1)

In [19]:
import numpy as np
import pyvista as pv

cloud = pv.PolyData(pcd_masked_down_pcdrgb[:, :3])
# Colorize the point cloud if it has color information
if pcd_masked_dowwn_color is not None:
    cloud["colors"] = pcd_masked_dowwn_color / 255  # Normalize the colors to [0, 1]

plotter = pv.Plotter()
plotter.add_points(cloud, scalars="colors")
plotter.show()


Widget(value="<iframe src='http://localhost:39695/index.html?ui=P_0x7ff66f3985b0_2&reconnect=auto' style='widt…

### Visualize `PCD` and `Voxel grid` at the same time   

In [16]:
import numpy as np
import pyvista as pv

cloud = pv.PolyData(pcd_masked[:, :3])
# cloud = pv.PolyData(pcd_masked_down[:, :3])

if point_cloud_data.shape[1] == 6:
    # Normalize the colors to [0, 1]
    cloud.point_arrays["colors"] = point_cloud_data[:, 3:] / 255
grid = pv.UnstructuredGrid()

for voxel in occupied_voxels:
    voxel = voxel * voxel_size
    cube = pv.Cube(center=voxel, x_length=voxel_size, y_length=voxel_size, z_length=voxel_size)
    grid = grid.merge(cube)

plotter = pv.Plotter()
# plotter.add_points(cloud, scalars="colors" if point_cloud_data.shape[1] == 6 else None) # Colorize the point cloud
plotter.add_points(cloud, color="grey")
plotter.add_mesh(grid, color="blue", opacity=0.8)

# Show the plot
plotter.show()


Widget(value="<iframe src='http://localhost:41691/index.html?ui=P_0x7fa0644688b0_8&reconnect=auto' style='widt…

### ETC: Voxelize `pointcloud` data: `nd.array` type

In [ ]:
import numpy as np

def point_cloud_to_voxel(point_cloud, voxel_size):
    # Calculate voxel grid dimensions
    min_coords = np.min(point_cloud, axis=0)
    max_coords = np.max(point_cloud, axis=0)
    grid_dims = np.ceil((max_coords - min_coords) / voxel_size).astype(int)

    # Create an empty voxel grid
    voxel_grid = np.zeros(grid_dims, dtype=np.uint8)

    # Iterate over points and set voxel occupancy
    for point in point_cloud:
        voxel_index = np.floor((point - min_coords) / voxel_size).astype(int)
        voxel_grid[tuple(voxel_index)] = 1

    return voxel_grid

voxel_size = 0.05
voxel_grid = point_cloud_to_voxel(pcd, voxel_size)